# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import torch
import logging
from transformers import AutoModelForCausalLM
from nnsight import LanguageModel

sys.path.append("../")
import src.utils.logging_utils as logging_utils
import src.functional as functional
import src.models as models
import src.tokens as tokens
import src.dataset as dataset
from src.models import ModelandTokenizer
import src.patchscope_eval as patchscope_eval
import proto.patchscope_pb2 as patchscope_pb2
import src.dataset_manager as dataset_manager

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.INFO,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")

2024-11-01 03:00:05 __main__ INFO     torch.__version__='2.5.0+cu124', torch.version.cuda='12.4'


In [3]:
# MODEL_KEY = "meta-llama/Llama-3.2-3B-Instruct"
MODEL_KEY = "meta-llama/Llama-3.1-8B-Instruct"
# MODEL_KEY = "meta-llama/Llama-3.2-3B"
# MODEL_KEY = "meta-llama/Llama-3.1-8B"

# MODEL_KEY = "google/gemma-2-9b-it"
# MODEL_KEY = "google/gemma-2-2b"

mt = models.ModelandTokenizer(
    model_key=MODEL_KEY,
    torch_dtype=torch.bfloat16,
)

mt.n_layer

2024-11-01 03:00:05 src.models WARNING  meta-llama/Llama-3.1-8B-Instruct not found in models
If not found in cache, model will be downloaded from HuggingFace to cache directory


2024-11-01 03:00:06 accelerate.utils.modeling INFO     We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2024-11-01 03:00:12 src.models INFO     loaded model <meta-llama/Llama-3.1-8B-Instruct> | size: 15316.516 MB | dtype: torch.bfloat16 | device: cuda:0


32

# Demo

In [4]:
# true_prompt = "The city of Paris is in the country of France."
# false_prompt = "The city of Paris is in the country of Italy."
# true_input, false_input = [tokens.prepare_input(p, mt) for p in (true_prompt, false_prompt)]

In [5]:
# # layers = [24]
# layers = list(range(3, 30))

# true_h, false_h = [
#     patchscope_eval.get_h_layers(mt, input_, layers) for input_ in (true_input, false_input)
# ]

In [6]:
# true_token, false_token = [mt.tokenizer.encode(t)[-1] for t in ("True", "False")]
# [mt.tokenizer.decode(t) for t in (true_token, false_token)]

In [7]:
# for prompt, h in zip((true_prompt, false_prompt), (true_h, false_h)):
#     print(prompt)
#     _, result_dict = functional.patchscope(
#         mt = mt, 
#         hs = patchscope_eval.get_h_with_target_layer(h, 3) if len(h) == 1 else h,
#         target_prompt = target_prompt,
#         interested_tokens = (true_token, false_token),
#         k = 5)
#     for t in (true_token, false_token):
#         print("   ", result_dict[t])


In [8]:
# gmt_dataset = dataset.GMTDataset.from_csv("cities.csv", few_shot=False).examples
# len(gmt_dataset)

# Base evaluation

In [4]:
base_prompt = '''The city of Tokyo is in Japan. This statement is: True
The city of Hanoi is in Poland. This statement is: False
placeholder placeholder placeholder placeholder placeholder This statement is: '''

instruct_prompt = '''<|start_header_id|>user<|end_header_id|>

True or false: placeholder placeholder placeholder placeholder placeholder<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

'''

target_prompt = {
    "meta-llama/Llama-3.2-3B-Instruct" : instruct_prompt,
    "meta-llama/Llama-3.1-8B-Instruct" : instruct_prompt,
    "meta-llama/Llama-3.2-3B" : base_prompt,
    "meta-llama/Llama-3.1-8B" : base_prompt,
}[MODEL_KEY]

model_short_name = {
    "meta-llama/Llama-3.2-3B-Instruct" : "llama_3b_instruct",
    "meta-llama/Llama-3.1-8B-Instruct" : "llama_8b_instruct",
    "meta-llama/Llama-3.2-3B" : "llama_3b",
    "meta-llama/Llama-3.1-8B" : "llama_8b",
}[MODEL_KEY]

In [ ]:
print(MODEL_KEY, model_short_name)

meta-llama/Llama-3.1-8B llama_8b


In [11]:
result_set_name = model_short_name + "__all_layers"
evaluation_results = patchscope_pb2.EvaluationResults()

for filename in dataset.GMT_DATA_FILES:
    examples = dataset.GMTDataset.simple_get_examples(filename)[:100]
    evaluation_config = patchscope_pb2.EvaluationConfig(
        model_key=MODEL_KEY,
        dataset=filename,
        target_prompt=target_prompt,
        label_to_token={ "1": "True", "0": "False" },
        patchscope_config=patchscope_pb2.PatchscopeConfig(
            source_layers=list(range(mt.n_layer)),
            target_layers=[]
        )
    )
    evaluation_runner = patchscope_eval.EvaluationRunner(evaluation_config, mt)
    evaluation_result = evaluation_runner.evaluate(examples)
    evaluation_result.result_set_name = result_set_name
    # evaluation_result = evaluation_runner.evaluate([(true_prompt, True), (false_prompt, False)])
    evaluation_results.results.append(evaluation_result)
    print(filename, evaluation_result.accuracy)

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:37<00:00,  1.57s/it]


sp_en_trans.csv 0.41


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:36<00:00,  1.57s/it]


neg_sp_en_trans.csv 0.42


...

common_claim_true_false.csv 0.54


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:32<00:00,  1.53s/it]


companies_true_false.csv 0.63


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:32<00:00,  1.52s/it]

counterfact_true_false.csv 0.57


In [12]:
with open(f"results/{result_set_name}.binpb", "wb") as f:
    f.write(evaluation_results.SerializeToString())

# Finetuned evaluation

In [4]:
# finetuned_model_key = "sensharma/Llama-3.2-3B_Talkative_Probe_Step-3000"
finetuned_model_key = "/home/local_arnab/Codes/Projects/talkative_probes/results/patchscope_save/Llama-3.2-3B/checkpoint-12000"
tuned_model = AutoModelForCausalLM.from_pretrained(
    finetuned_model_key, torch_dtype=torch.bfloat16
).to("cuda")

tuned_lm = LanguageModel(tuned_model)
decoder_mt = ModelandTokenizer(
    base_lm = tuned_lm,
    tokenizer = mt.tokenizer
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  1.55it/s]


2024-10-30 19:37:23 src.models INFO     loaded model <EleutherAI/gpt-j-6B> | size: 6127.841 MB | dtype: torch.bfloat16 | device: cuda:0


In [10]:
decoder_mt.name = "Llama-3.2-3B/checkpoint-12000"

In [17]:
dataset_group = "geometry_of_truth"
(dataset,) = dataset_manager.DatasetManager.from_dataset_group(dataset_group).split([0.01])

In [18]:
len(dataset)

130

In [19]:
next(iter(dataset))

[ContextQASample(context='Peasants originated in Great Britain and were later introduced in Europe.', questions=['# Do you find this to be truthful?', '# Would you validate this statement?', '# Have I understood this correctly?', '# Do you agree that this is true?', '# May I assume this is correct?', '# Can you confirm the accuracy of this?', '# Is this statement true?', '# Are you confirming this is true?', '# Could you tell me if this is factual?', '# Do the facts support this statement?'], answers=['No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No'])]

In [20]:
from src.tokens import prepare_input, find_token_range

prompts = [
    # "The land of",
    # "The capital of France is",
    # "This is a"
    "The city of Paris is in the country of"
]

batch_inputs = prepare_input(
    tokenizer=mt,
    prompts=prompts,
    padding_side="left",
    # padding="max_length",
    # max_length=20,
    truncation=True,
    return_offsets_mapping=True
)

batch_inputs

{'input_ids': tensor([[128000,    791,   3363,    315,  12366,    374,    304,    279,   3224,
            315]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0'), 'offset_mapping': tensor([[[ 0,  0],
         [ 0,  3],
         [ 3,  8],
         [ 8, 11],
         [11, 17],
         [17, 20],
         [20, 23],
         [23, 27],
         [27, 35],
         [35, 38]]], device='cuda:0')}

In [21]:
evaluation_config = patchscope_pb2.EvaluationConfig(
    model_key=MODEL_KEY,
    dataset=dataset_group,
    patchscope_config=patchscope_pb2.PatchscopeConfig(
        source_layers=[24],
        target_layers=list(range(decoder_mt.n_layer))
    ),
    decoder_config=patchscope_pb2.DecoderConfig(
        name=finetuned_model_key
    )
)
evaluation_runner = patchscope_eval.EvaluationRunner(evaluation_config, mt, decoder_mt)
evaluation_result = evaluation_runner.evaluate(dataset)
print(evaluation_result.accuracy)

  0%|          | 0/130 [00:00<?, ?it/s]

100%|██████████| 130/130 [00:50<00:00,  2.58it/s]

0.6692307692307692


In [34]:
# prompt = "The boys are playing in the park. This statement is in the present tense."
prompt = "Grace is a school teacher. Would you say this statement is about a male?"

functional.predict_next_token(decoder_mt, prompt, k=10)

[[PredictedToken(token=' Yes', prob=0.5310860276222229, logit=21.25, token_id=7566),
  PredictedToken(token=' No', prob=0.46868178248405457, logit=21.125, token_id=2360),
  PredictedToken(token='Yes', prob=6.976826989557594e-05, logit=12.3125, token_id=9642),
  PredictedToken(token=' yes', prob=4.79509835713543e-05, logit=11.9375, token_id=10035),
  PredictedToken(token=' YES', prob=2.7321648303768598e-05, logit=11.375, token_id=14410),
  PredictedToken(token='No', prob=7.353521596087376e-06, logit=10.0625, token_id=2822),
  PredictedToken(token=' Yeah', prob=6.489459792646812e-06, logit=9.9375, token_id=22335),
  PredictedToken(token='.No', prob=3.6975827697460772e-06, logit=9.375, token_id=17184),
  PredictedToken(token='yes', prob=3.2631053272780264e-06, logit=9.25, token_id=9891),
  PredictedToken(token='-No', prob=2.87968032353092e-06, logit=9.125, token_id=99076)]]

In [29]:
inputs = prepare_input(
    tokenizer=decoder_mt,
    prompts=[prompt],
    padding_side="left",
)

output = decoder_mt._model.generate(
    **inputs, do_sample = False
)

mt.tokenizer.decode(output[0], skip_special_tokens=True)  # Decode the output

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'The city of Paris is in the country of France Yes Yes Yes Yes Yes Yes Yes Yes Yes'

In [39]:
from src.functional import get_batch_concept_activations

prompts = [
    "Grace is a school teacher",
]

latents = get_batch_concept_activations(
    mt=mt,
    prompts=prompts,
    interested_layer_indices=list(range(5, 20)),
    check_prediction=None,
    on_token_occur=None,
)

In [45]:
latents[0].questions = ["# Would you say this statement is about a male?"]
latents[0].answers = [" No"]

In [61]:
from src.activation_manager import ActivationSample
import random

add_to_buffer = []
for latent_cache in latents:
    for layer_name in latent_cache.latents.keys():
        activation = latent_cache.latents[layer_name]
        # query, label = self.get_latent_qa(
        #     correct_label=latent_cache.correct_label,
        #     wrong_label=latent_cache.incorrect_label,
        #     group=latent_cache.group,
        # )
        question, label = random.choice(
            list(zip(latent_cache.questions, latent_cache.answers))
        )
        add_to_buffer.append(
            ActivationSample(
                activation=activation,
                context=latent_cache.context,
                question=question,
                label=label,
                layer_name=layer_name,
            )
        )

In [62]:
from src.train_utils import evaluate

evaluate(decoder_mt, add_to_buffer)  # Evaluate the collected sample

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/nnsight/tracing/Node.py:380: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = self.target(*args, **kwargs)
Evaluating: 100%|██████████| 1/1 [00:00<00:00,  4.35it/s]


0.0

In [63]:
ds = dataset_manager.DatasetManager.from_named_datasets(
    [("md_gender", "md_gender")],
    batch_size=5
)

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/datasets/load.py:1486: FutureWarning: The repository for facebook/md_gender_bias contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/facebook/md_gender_bias
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [56]:
batch = next(iter(ds))

In [60]:
batch[0].__dict__

{'context': "I'm a football fan that knows he plays for FC Tamboc from Arsenal.\n\nThis text is about Vladislav Ryzhkov.",
 'questions': ['# Am I reading about someone who identifies as female?',
  '# Is the person mentioned male?',
  '# Does this portray a male individual?',
  '# Is the subject of this sentence male?',
  '# Does this content reference a male person?',
  '# Is the character described here female?',
  '# Am I learning about a male individual here?',
  '# Is this about someone who is male?',
  '# Is the focus of this sentence on someone male?',
  '# Does this refer to a female individual?'],
 'answers': ['No',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'No']}

# Demo 2

In [4]:
prompts = [
    "placeholder placeholder placeholder The city of Paris is in the country of",
    "placeholder Hello world"
]
input = functional.prepare_input(prompts, mt, return_offsets_mapping=True)
input.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping'])

In [33]:
input.input_ids.shape

torch.Size([2, 13])

In [39]:
input.input_ids

tensor([[128000,  12665,   6002,   6002,    578,   3363,    315,  12366,    374,
            304,    279,   3224,    315],
        [128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
         128000,  12665,  22691,   1917]], device='cuda:0')

In [55]:
bools = torch.argwhere((input.input_ids == 12665) | (input.input_ids == 6002))
bools

tensor([[ 0,  1],
        [ 0,  2],
        [ 0,  3],
        [ 1, 10]], device='cuda:0')

In [53]:
bools.shape

torch.Size([2, 13])

In [59]:
(input.input_ids == 12665).dtype

torch.bool

In [54]:
input.offset_mapping[bools]

tensor([[ 0, 11],
        [11, 23],
        [23, 35],
        [ 0, 11]], device='cuda:0')

In [63]:
mt.tokenizer.encode(" No")

[128000, 2360]

In [64]:
mt.tokenizer.encode(" Yes")

[128000, 7566]

In [35]:
mt.tokenizer.encode("placeholder ")

[128000, 12665, 220]

In [23]:
input.offset_mapping.shape

torch.Size([2, 10, 2])

In [14]:
hs = functional.get_hs(mt, input=input, locations=[(mt.layer_names[-1], -1), (mt.layer_names[-2], -1)])

In [19]:
hs[('model.layers.26', -1)].shape

torch.Size([2, 3072])

In [62]:
logits = hs[('model.layers.27', -1)]
logits.shape

torch.Size([2, 3072])

In [5]:
source_prompts = [
    "The quick brown fox jumps over the lazy dog.",
    "Hello my name is Daniel."
]
source_hs = patchscope_eval.get_source_hs(mt, source_prompts, [24,25])

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Could not load library libnvrtc.so.12. Error: libnvrtc.so.12: cannot open shared object file: No such file or directory
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [17]:
result = functional.patchscope(mt, source_hs, prompts)
result

placeholder position: 1 | token: placeholder
placeholder position: 1 | token: placeholder
placeholder position: 2 | token:  placeholder
placeholder position: 2 | token:  placeholder
placeholder position: 3 | token:  placeholder
placeholder position: 3 | token:  placeholder
placeholder position: 10 | token: placeholder
placeholder position: 10 | token: placeholder


tensor([[[ 7.5312,  6.7812,  5.2500,  ..., -2.5938, -2.5938, -2.5938]],

        [[16.8750,  6.5938,  7.0938,  ..., -4.4062, -4.4062, -4.4062]]],
       device='cuda:0')

In [15]:
result[:,0,2360]

tensor([9.0625, 8.1250], device='cuda:0')

In [18]:
mt.tokenizer.decode(2360)

' No'

In [16]:
result[:,0,7566]

tensor([7.5938, 6.1875], device='cuda:0')

In [20]:
result.squeeze().take_along_dim(torch.tensor([[2360, 7566]]).cuda(), dim=-1)

tensor([[9.0625, 7.5938],
        [8.1250, 6.1875]], device='cuda:0')

In [24]:
true_prompt = "The city of Paris is in the country of France."
false_prompt = "The city of Paris is in the country of Italy."
source_prompts = [true_prompt, false_prompt]
target_prompts = [target_prompt, target_prompt]
print(source_prompts)
print(target_prompts)

['The city of Paris is in the country of France.', 'The city of Paris is in the country of Italy.']
['<|start_header_id|>user<|end_header_id|>\n\nTrue or false: placeholder placeholder placeholder placeholder placeholder<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n\n', '<|start_header_id|>user<|end_header_id|>\n\nTrue or false: placeholder placeholder placeholder placeholder placeholder<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n\n']


In [28]:
patchscope_runner = patchscope_eval.PatchscopeRunner(
    mt,
    None,
    patchscope_pb2.PatchscopeConfig(
        source_layers=list(range(mt.n_layer)),
        target_layers=[],
    ),
    interested_tokens=[" False", " True"]
)

In [39]:
patchscope_runner.run([source_prompts[1]], [target_prompts[1]])

[' False']

# Base evaluation 2

In [4]:
dataset_group = "geometry_of_truth"
dataset = dataset_manager.DatasetManager.from_dataset_group(dataset_group, batch_size=32)
len(dataset)

220

In [5]:
prompt_format = '''<|start_header_id|>user<|end_header_id|>

{} Please answer yes or no.<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

'''

In [10]:
evaluation_config = patchscope_pb2.EvaluationConfig(
    model_key=MODEL_KEY,
    dataset=dataset_group,
    patchscope_config=patchscope_pb2.PatchscopeConfig(
        source_layers=[24],
        target_layers=list(range(mt.n_layer))
    ),
    interested_tokens=["Yes", "No"],
    prompt_format=prompt_format,
)
evaluation_runner = patchscope_eval.EvaluationRunner(evaluation_config, mt)
evaluation_result = evaluation_runner.evaluate(dataset, max_examples=1000)
print(evaluation_result.accuracy)

 14%|███████████▍                                                                     | 31/220 [01:20<08:10,  2.59s/it]

0.69921875
